In [1]:
from utils import *
import array 

from pydub import AudioSegment
import tensorflow as tf

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, Flatten, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPool2D, concatenate, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.python.keras.utils import to_categorical

from keras.layers import Input, GRU, RepeatVector, BatchNormalization, TimeDistributed, Conv1D
from keras.layers import GlobalAveragePooling1D, LSTM, MaxPooling1D, CuDNNLSTM, Bidirectional
from keras import backend as K
from keras.layers import  Conv2D, MaxPooling2D, UpSampling2D, Lambda, Reshape
import keras
from keras.layers import AveragePooling1D

from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numba
from keras import losses
import dask.dataframe as dd
from dask.multiprocessing import get
from keras.layers import Embedding, Flatten, Input, merge

%matplotlib inline

Using TensorFlow backend.


In [2]:
all_labels = [x[0].split('/')[-1] for x in os.walk("data/train/audio/")]
 


exclusions = ["","_background_noise_"]
POSSIBLE_LABELS = [item for item in all_labels if item not in exclusions]

In [3]:
# POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}
len(id2name)

32

#### create validation dataframe

In [4]:
train_df = pickle.load( open("cache/train_df_all_labels.pik","rb"))
valid_df = pickle.load( open("cache/valid_df_all_labels.pik","rb"))
silent_df = pickle.load(open("cache/silent_df.pik","rb"))

In [5]:
train_df["label_user"] = train_df["label"]+train_df["user_id"]
valid_df["label_user"] = valid_df["label"]+valid_df["user_id"]

In [63]:
def train_generator(train_batch_size):
    
    
    #choose records which have same speaker repeating them more than once 
    tmp = train_df.groupby("label_user").filter(lambda x: len(x) > 1) 
    tmp.reset_index(inplace=True,drop=True)
    while True:
        
        #choose a random label, utterances should be from same label
        this_train = tmp[tmp.label_id==np.random.randint(0,len(id2name))] 
    
        #group by distinct speakers for this label
        g = this_train.groupby("label_user")
        all_keys = list(g.groups.keys())
        
        if not all_keys:
            continue      
        
        for _ in range(0, 2): #almost take from every pair in this label
            
            x_batch_A_1 = []
            x_batch_A_2 = []
            x_batch_B = []
        
       
            
            for i in range(train_batch_size):
                
                #choose a random speaker A
                i = np.random.choice(range(len(all_keys)))
                similars = g.get_group(all_keys[i])
                
                #choose 2 distinct utterances from speaker A 
                sim_1, sim_2 = np.random.choice(range(len(similars)),2,replace=False)
                A_1 , A_2 = similars.raw.iloc[sim_1], similars.raw.iloc[sim_2] 
                
                # choose a different speaker B
                possible_diffs = range(len(all_keys)-1) if (i == len(all_keys)-1) else range(len(all_keys))
                diff_i = np.random.choice(possible_diffs) 
                diff_i = diff_i+1 if diff_i >=i else diff_i
                diff_i = min(diff_i, len(all_keys)-1)
                diffs = g.get_group(all_keys[diff_i])
                
                #choose a random utternace from B's records
                B = diffs.raw.iloc[np.random.choice(range(len(diffs)))]
                
                
                # create triplet
                x_batch_A_1.append(A_1.T)
                x_batch_A_2.append(A_2.T)
                x_batch_B.append(B.T)
#                 x_batch.append([A_1.T,A_2.T,B.T])
                
            x_batch_A_1 = 1.- np.array(x_batch_A_1)/-80.
            x_batch_A_2 = 1.- np.array(x_batch_A_2)/-80.
            x_batch_B = 1.- np.array(x_batch_B)/-80.
        
            y_batch = np.ones((train_batch_size)) 
            
            yield [x_batch_A_1,x_batch_A_2,x_batch_B], y_batch# [y_batch,y_batch,y_batch]

In [32]:
t = train_generator(32)

tt = next(t)[0]

tt[0].mean(), tt[1].mean(), tt[2].mean(), 

(0.31655370011274542, 0.31404827090262433, 0.37890711528615456)

In [33]:

tt[1].shape

(32, 32, 256)

In [64]:
def valid_generator(train_batch_size):
    
    
    #choose records which have same speaker repeating them more than once 
    tmp = valid_df.groupby("label_user").filter(lambda x: len(x) > 1) 
    tmp.reset_index(inplace=True,drop=True)
    
    while True:
        
        #choose a random label, utterances should be from same label
        this_train = tmp[tmp.label_id==np.random.randint(0,len(id2name))] 
    
        #group by distinct speakers for this label
        g = this_train.groupby("label_user")
        all_keys = list(g.groups.keys())
        
        if not all_keys:
            continue 
            
            
        for _ in range(0, 2): #for _ in range(0, len(all_keys)): #almost take from every pair in this label
            
            x_batch_A_1 = []
            x_batch_A_2 = []
            x_batch_B = []
            
            for i in range(train_batch_size):
                
                #choose a random speaker A
                i = np.random.choice(range(len(all_keys)))
                similars = g.get_group(all_keys[i])
                
                #choose 2 distinct utterances from speaker A 
                sim_1, sim_2 = np.random.choice(range(len(similars)),2,replace=False)
                A_1 , A_2 = similars.raw.iloc[sim_1], similars.raw.iloc[sim_2] 
                
                # choose a different speaker B
                possible_diffs = range(len(all_keys)-1) if (i == len(all_keys)-1) else range(len(all_keys))
                diff_i = np.random.choice(possible_diffs) 
                diff_i = diff_i+1 if diff_i >=i else diff_i
                diff_i = min(diff_i, len(all_keys)-1)
                diffs = g.get_group(all_keys[diff_i])
                
                #choose a random utternace from B's records
                B = diffs.raw.iloc[np.random.choice(range(len(diffs)))]
                
                
                # create triplet
                x_batch_A_1.append(A_1.T)
                x_batch_A_2.append(A_2.T)
                x_batch_B.append(B.T)
#                 x_batch.append([A_1.T,A_2.T,B.T])
                
            x_batch_A_1 = 1.- np.array(x_batch_A_1)/-80.
            x_batch_A_2 = 1.- np.array(x_batch_A_2)/-80.
            x_batch_B = 1.- np.array(x_batch_B)/-80.
        
            y_batch = np.ones((train_batch_size)) 
            
            yield x_batch_A_1,x_batch_A_2,x_batch_B, y_batch #[y_batch,y_batch,y_batch]

In [65]:
v = valid_generator(64)


In [66]:
%time next(v)[0][0].shape

CPU times: user 512 ms, sys: 0 ns, total: 512 ms
Wall time: 506 ms


(32, 256)

In [67]:
X_1,X_2,X_3, y_valid = [], [],[], []

for i in range( 400):
    x1,x2,x3, y_v = next(v)
    X_1.append(x1) 
    X_2.append(x2) 
    X_3.append(x3) 
    
    y_valid.append(y_v)
    

In [68]:
X_1 = np.vstack(X_1)
X_2 = np.vstack(X_2)
X_3 = np.vstack(X_3)
y_valid = np.hstack(y_valid)

In [69]:
X_1.shape, y_valid.shape

((25600, 32, 256), (25600,))

### Model


In [70]:
def batch_relu(x):
    x = BatchNormalization()(x)    
    x = Activation('relu')(x)
    
    return x 

In [71]:
timesteps, input_dim , latent_dim = 32,256, 128
p = .0

In [8]:


# def triplet_loss(y_true, y_pred):
#     mse1 = losses.mean_squared_error(y_pred[0],y_pred[1]) #K.sqrt(y_pred[0] - y_pred[1]).mean(axis=-1)
#     mse2 = losses.mean_squared_error(y_pred[0],y_pred[2])#K.sqr(y_pred[0] - y_pred[2]).mean(axis=-1)
    
#     return K.maximum(0., mse1 - mse2 + 1) - y_true[0]*0


# def triplet_loss( y_true, y_pred):

#     embeddings = K.reshape(y_pred, (-1, 3, latent_dim))

#     positive_distance = K.mean(K.square(embeddings[:,0] - embeddings[:,1]),axis=-1)
#     negative_distance = K.mean(K.square(embeddings[:,0] - embeddings[:,2]),axis=-1)
#     return K.mean(K.maximum(0.0, positive_distance - negative_distance + 1.0))

In [72]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)



def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss



In [88]:

def embedder(filter_size=2):
    v_input = Input(shape=( timesteps,input_dim))
    x = BatchNormalization()(v_input)
    
    x =  Reshape((timesteps, input_dim,1))(x)
    
#     x = Conv1D(64,filter_size,padding='same')(x)
#     x = batch_relu(x)
#     x = Conv1D(64,filter_size,padding='same')(x)
#     x = batch_relu(x)
    
    
#     x  = MaxPooling1D(2)(x) 
    
#     x = Conv1D(128,filter_size,padding='same')(x)
#     x = batch_relu(x)
#     x = Conv1D(128,filter_size,padding='same')(x)
#     x = batch_relu(x)

#     x  = AveragePooling1D(2)(x)   
    
# #     x = Bidirectional(CuDNNLSTM(128,return_sequences=False))(x)
# #     x = BatchNormalization()(x)
# #     x = Dense(128,activation='relu')(x)
    
#     x = GlobalMaxPooling1D()(x)
    
    
    
    
    
    x = Conv2D(32,(7,7),padding='same')(x) #was 32
    x = batch_relu(x)

    x = Conv2D(64,(3,3),padding='same')(x)
    x = batch_relu(x)

    
    x = MaxPooling2D((1,3))(x)
    
    x = Conv2D(64,(3,3),padding='same')(x)
    x = batch_relu(x)
    x = Conv2D(64,(3,3),padding='same')(x)
    x = batch_relu(x)

    
    
    x = MaxPooling2D((2,3))(x)
    

    x = Conv2D(128,(3,3),padding='same')(x)
    x = batch_relu(x)
    x = Conv2D(128,(3,3),padding='same')(x)
    x = batch_relu(x)

    x = MaxPooling2D((1,3))(x)
    
    
    x = Conv2D(128,(3,3),padding='same')(x)
    x = batch_relu(x)
    x = Conv2D(128,(3,3),padding='same')(x)
    x = batch_relu(x)
    print int(x.shape[-1]) * int(x.shape[-2])
    x = Reshape((16,9*128))(x)
    
    x = Bidirectional(CuDNNLSTM(128,return_sequences=False))(x)
    
    

    return Model(v_input, x)

In [89]:


xA1 = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]
xA2 = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]
xB = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]

# xA1_freq = Reshape((input_dim, timesteps))(xA1)
# xA2_freq = Reshape((input_dim, timesteps))(xA2)
# xB_freq = Reshape((input_dim, timesteps))(xB)



embedding = embedder(5)

xA1_freq = embedding(xA1)
xA2_freq = embedding(xA2)
xB_freq = embedding(xB)

loss = merge(
    [xA1_freq, xB_freq, xA2_freq],
    mode=bpr_triplet_loss,
    name='loss',
    output_shape=(1,))

model = Model(
        inputs=[xA1, xA2, xB],
        outputs=loss) #concatenate([xA1_freq,xA2_freq,xB_freq],axis=0)

model.compile(Adam(lr=1e-3),identity_loss)

1152


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [90]:
embedding.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 32, 256)           0         
_________________________________________________________________
batch_normalization_38 (Batc (None, 32, 256)           1024      
_________________________________________________________________
reshape_22 (Reshape)         (None, 32, 256, 1)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 256, 32)       1600      
_________________________________________________________________
batch_normalization_39 (Batc (None, 32, 256, 32)       128       
_________________________________________________________________
activation_29 (Activation)   (None, 32, 256, 32)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 256, 64)       18496     
__________

In [91]:
exp_name = "speaker_embedding_fixedval_conv2dlstm" 
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1),
             
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=0.01,
                              min_lr=1e-5),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/{}.hdf5'.format(exp_name),
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')            ]

In [92]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 32, 256)      0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           (None, 32, 256)      0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           (None, 32, 256)      0                                            
__________________________________________________________________________________________________
model_12 (Model)                (None, 256)          1927296     input_37[0][0]                   
                                                                 input_38[0][0]                   
          

freqconv

Epoch 15/100
905/905 [==============================] - 94s 104ms/step - loss: 0.0247 - acc: 0.0245 - val_loss: 0.0230 - val_acc: 0.0236


timeconv wlstm
Epoch 17/100
905/905 [==============================] - 84s 92ms/step - loss: 0.1163 - val_loss: 0.1131

In [78]:
# model.load_weights("weights/speaker_embedding_timeconv_wlstm_c2.hdf5")
model.load_weights("weights/{}.hdf5".format(exp_name))

In [93]:
batch_size = 64

history = model.fit_generator(generator=train_generator(batch_size),
                              steps_per_epoch=train_df.shape[0]//batch_size,
                              epochs=100,
                              callbacks=callbacks,
#                               validation_data=valid_generator(batch_size),
#                               validation_steps=int(np.ceil(valid_df.shape[0]//batch_size)))
                              validation_data=([X_1,X_2,X_3],y_valid))#


Epoch 1/100
905/905 [==============================] - 881s 974ms/step - loss: 0.0754 - val_loss: 0.0697
Epoch 2/100
905/905 [==============================] - 871s 962ms/step - loss: 0.0431 - val_loss: 0.0634
Epoch 3/100
905/905 [==============================] - 870s 962ms/step - loss: 0.0351 - val_loss: 0.0490
Epoch 4/100
905/905 [==============================] - 870s 962ms/step - loss: 0.0324 - val_loss: 0.0454
Epoch 5/100
905/905 [==============================] - 872s 963ms/step - loss: 0.0286 - val_loss: 0.0364
Epoch 6/100
905/905 [==============================] - 871s 963ms/step - loss: 0.0266 - val_loss: 0.0357
Epoch 7/100
905/905 [==============================] - 872s 964ms/step - loss: 0.0247 - val_loss: 0.0329
Epoch 8/100
905/905 [==============================] - 870s 961ms/step - loss: 0.0234 - val_loss: 0.0368
Epoch 9/100
904/905 [============================>.] - ETA: 0s - loss: 0.0205
Epoch 00009: reducing learning rate to 0.00010000000475.
905/905 [================

KeyboardInterrupt: 

In [94]:
model.load_weights("weights/{}.hdf5".format(exp_name))

In [95]:
embedding.save_weights("weights/speaker_embedding_conv2dlstm.hdf5")

In [96]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

0.996727
0.00337073
0.990294
-0.000155765
0.991019
0.00466726
1.00052
-0.00549822
0.990437
-0.00571059
0.984998
-0.00494674
0.982686
-0.00640985
0.989239
-0.000872991
1.0419
0.00901419


In [49]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

0.821782
0.00257948
0.97807
-0.0011363
0.990386
0.00306028
0.995768
0.00208017
1.00084
-0.00353018
1.10434
0.0176663


In [51]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

0.82218
0.00224655
0.978326
-0.000955906
0.990576
0.00329765
0.99577
0.00218155
1.00076
-0.00361477
1.10443
0.0177416


In [47]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

1.0
-0.000116561
1.0
-0.000564231
1.0
0.00077817
1.0
-6.0897e-05
1.0
0.000275884
1.0
-0.000213044
